In [1]:
import torch
import torch.nn.functional as F
from torch import optim

import yaml
import pandas as pd
import numpy as np
import pickle

from sklearn.preprocessing import quantile_transform 
from x_transformers import XTransformer, TransformerWrapper, Decoder, Encoder, ViTransformerWrapper

from aptamer_transformer.model import *
from aptamer_transformer.factories_model_loss import *
from aptamer_transformer.data_utils import *

from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.trainers import BpeTrainer
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.processors import TemplateProcessing
from transformers import PreTrainedTokenizerFast
from transformers import DataCollatorForLanguageModeling
from transformers import AutoTokenizer, DataCollatorForLanguageModeling
from tokenizers.pre_tokenizers import PreTokenizer


%load_ext autoreload
%autoreload 2

2024-01-18 13:55:54.310258: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-18 13:55:54.310324: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-18 13:55:54.311378: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-18 13:55:54.317641: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-18 13:55:55.266162: W tensorflow/compiler/tf2

In [ ]:
df = pd.read_pickle('../data/saved_processed_data/saved_h5/seq_struct_n_classes_2.pkl')


In [ ]:
structures = df.dot_bracket_struc.apply(lambda x: ' '.join(x)).values.tolist()
seqs = df.Sequence.apply(lambda x: ' '.join(x)).values.tolist()

seq_structs = [f'{seq}{struct}' for seq, struct in zip(df.Sequence, df.dot_bracket_struc)]

seq_structs_white_space = [(seq , struct) for seq, struct in zip(seqs, structures)]

len_seq_structs = [len(struct) for struct in seq_structs]
print(max(len_seq_structs))

In [24]:
cfg = read_cfg('../aptamer_transformer/config.yaml')

dna_dataset = load_dataset(cfg)

In [25]:
dna_dataset.tokenized_seq_struc

tensor([[ 2,  8, 11,  ...,  7,  7,  4],
        [ 2, 11,  9,  ...,  4,  0,  0],
        [ 2,  8, 10,  ...,  4,  0,  0],
        ...,
        [ 2, 10, 10,  ...,  4,  0,  0],
        [ 2, 11, 10,  ...,  4,  0,  0],
        [ 2, 11, 10,  ...,  4,  0,  0]])

In [11]:
class Object(object):
    pass

args = Object()
args.distributed = False
train_loader, val_loader, test_loader, train_sampler = get_data_loaders(dna_dataset, cfg, args)


In [13]:
batch = next(iter(train_loader))

In [23]:
np.array(batch[0]).T

array([['A G G A T T A T C G T C C G G G G G G G G G G C C C G G T T T C G G G G T C G T',
        '. . . . . . . . . . . ( ( ( ( ( . . . . . . . ) ) ) ) ) . . . ( ( . . . . ) ) .'],
       ['G G G G G G G G G G G C G G G C T G G C T T T A T T C C G G T T T C G T G T T T',
        '. . . . . . . . . . . ( ( ( . . . . . . . . . . . . ) ) ) . . . . . . . . . . .'],
       ['G G G C G G G C C C C C C C C G C A A G G G G G T C G T T T C T A C A G T G G G',
        '( ( ( . . . . ) ) ) ( ( ( ( ( . . . . ) ) ) ) ) . . . . . . . . . . . . . . . .'],
       ...,
       ['G G G G G G G C G C G C C C C G G G G C T A G G T G T A C G C T C G G T T T A G',
        '. . ( ( ( ( . . . . . ) ) ) ) . . . . . . . . . . . . . . . . . . . . . . . . .'],
       ['G G G G C C C C C C C C C C C C C G C C C A C T C T C G G T G A C G C A C A G C',
        '( ( ( ( . . . . . ) ) ) ) . . ( ( ( . . . . . . . . ) ) ) ( ( . . . . . ) ) . .'],
       ['G G G C G G G A G G G A G G G G G G G G G T G C C G C G T C T G 

In [4]:
dna_dataset.tokenizer.batch_decode(dna_dataset.tokenized_seq_struc[0:10])

['[CLS] A T C G G G G G G T G G G T T T G G G T A T G G G G T C T G C A C T A T G G C T C [SEP]............................. ( (...... ) ).. [SEP]',
 '[CLS] T C G C G G G G G G C G G G T C G G G T G C T C G T T C G A G G G G T C G C A G [SEP]. (. ( (..... ) ). ) ( ( (.......... ) ) ).......... [SEP] [PAD] [PAD]',
 '[CLS] A G G T T G G T G T A G G T T T A G G T T G T A T T G G T T A G G G T T G A G A [SEP]........................................ [SEP] [PAD] [PAD]',
 '[CLS] T G G C T G T C T T A C G A T C G C G A A G G A C G A T T G A A A A C T T T G A [SEP]............. ( ( ( (....... ) ) ) )............ [SEP] [PAD] [PAD]',
 '[CLS] G G G G G G G G G G G G G G G G G G G G G G G G G G G G G G G G G G G G G G G G [SEP]........................................ [SEP] [PAD] [PAD]',
 '[CLS] A T A G T T A C G G G T G A G C C G T C A T T C A A G T T T A C T A C T T C T G [SEP]...... ( ( ( (..... ) ) ) )..... ( ( ( (..... ) ) ) )... [SEP] [PAD] [PAD]',
 '[CLS] C C C A T G G T A G G T A T T G C T T

In [ ]:
tokenizer = Tokenizer(BPE(unk_token="N"), )
special_tokens = ["[PAD]", "N", "[CLS]", "[MASK]", "[SEP]"]
tokenizer.add_special_tokens(special_tokens)

tokenizer.pre_tokenizer = Whitespace()
tokenizer.mask_token = "[MASK]"
tokenizer.cls_token = "[CLS]"
tokenizer.pad_token = "[PAD]"
tokenizer.unknown_token = "N"
tokenizer.sep_token = "[SEP]"
tokenizer.model_max_length = 85
tokenizer.enable_padding(pad_id=0, pad_token="[PAD]")

# Post processor
tokenizer.post_processor = TemplateProcessing(
    single="[CLS] $A",
    pair="[CLS] $A [SEP] $B:1 [SEP]:1",
    special_tokens=[("[PAD]", 0), ("N", 1), ("[CLS]", 2), ("[MASK]", 3), ("[SEP]", 4)]
)

# Train the tokenizer
trainer = BpeTrainer(special_tokens=special_tokens)
tokenizer.train_from_iterator(seq_structs_white_space, trainer=trainer)

# Create a fast tokenizer
fast_tokenizer = PreTrainedTokenizerFast(
    tokenizer_object=tokenizer, 
    model_max_length=85, 
    sep_token="[SEP]", 
    cls_token="[CLS]", 
    unk_token="N", 
    pad_token="[PAD]", 
    mask_token="[MASK]", 
    return_special_tokens_mask=1
)

fast_tokenizer

In [ ]:
fast_tokenizer.save_pretrained('../data/tokenizers/seq_struct_sep_whitespace')

In [ ]:
cfg = read_cfg('../aptamer_transformer/config.yaml')
# Initialize the model and optimizer here as you did during training
device = torch.device("cuda:0")  
cfg.update({
    'device': device,
})

model = get_model(cfg).to(device)